# Library

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.filterwarnings(action='ignore')

# GPU 용량 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 최적 모델 찾기 (seed = 42)
- [최적 모델 찾기](https://snowdeer.github.io/machine-learning/2018/01/09/find-best-model/)
- 모델 생성 시 seed = 42 ~ 50일 때, 각각의 best모델에서 data seed = 1~50에서의 평균 정확도
- 모델 생성 시 seed = 42, 평균 정확도 : 82.6
- 모델 생성 시 seed = 43, 평균 정확도 : 81.6
- 모델 생성 시 seed = 44, 평균 정확도 : 82.5
- 모델 생성 시 seed = 45, 평균 정확도 : 82.5
- 모델 생성 시 seed = 46, 평균 정확도 : 81.6
- 모델 생성 시 seed = 47, 평균 정확도 : 81.8
- 모델 생성 시 seed = 48, 평균 정확도 : 81.1
- 모델 생성 시 seed = 49, 평균 정확도 : 82.3
- 모델 생성 시 seed = 50, 평균 정확도 : 80

## seed별 best model 저장

In [43]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 50
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 50 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed'f'{seed_num}''-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6754 - acc: 0.5949 - val_loss: 0.6753 - val_acc: 0.5970

Epoch 00001: val_loss improved from inf to 0.67535, saving model to ./model/seed50-01-0.6753.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6743 - acc: 0.6031 - val_loss: 0.6701 - val_acc: 0.5970

Epoch 00002: val_loss improved from 0.67535 to 0.67009, saving model to ./model/seed50-02-0.6701.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6285 - acc: 0.6398 - val_loss: 0.5877 - val_acc: 0.7025

Epoch 00003: val_loss improved from 0.67009 to 0.58767, saving model to ./model/seed50-03-0.5877.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.5010 - acc: 0.7681 - val_loss: 0.5157 - val_acc: 0.7536

Epoch 00004: val_loss improved from 0.58767 to 0.51568, saving model to ./model/seed50-04-0.5157.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

In [44]:
# ----------------------------
seed_num = 50
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/seed50-04-0.5157.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7870550161812297, seed_num = 0
정확도 :0.8019417475728156, seed_num = 1
정확도 :0.7954692556634304, seed_num = 2
정확도 :0.8090614886731392, seed_num = 3
정확도 :0.7954692556634304, seed_num = 4
정확도 :0.8025889967637541, seed_num = 5
정확도 :0.7954692556634304, seed_num = 6
정확도 :0.7877022653721683, seed_num = 7
정확도 :0.8084142394822007, seed_num = 8
정확도 :0.8006472491909385, seed_num = 9
정확도 :0.8129449838187702, seed_num = 10
정확도 :0.8129449838187702, seed_num = 11
정확도 :0.8064724919093851, seed_num = 12
정확도 :0.8116504854368932, seed_num = 13
정확도 :0.8045307443365696, seed_num = 14
정확도 :0.8019417475728156, seed_num = 15
정확도 :0.8116504854368932, seed_num = 16
정확도 :0.7902912621359224, seed_num = 17
정확도 :0.8006472491909385, seed_num = 18
정확도 :0.8019417475728156, seed_num = 19
정확도 :0.8071197411003236, seed_num = 20
정확도 :0.7909385113268609, seed_num = 21
정확도 :0.7941747572815534, seed_num = 22
정확도 :0.7922330097087379, seed_num = 23
정확도 :0.7967637540453074, seed_num = 24
정확도 :0.8148867313915857, seed_num =

# 함수 모음

## val_acc / LSTM_notallfit_early(name, ep)

In [21]:
## 수정 

from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

import random
import sklearn
from tqdm.notebook import tqdm

def LSTM_notallfit_early(name,ep):
    with tf.device('/device:GPU:0'):
        
        # 1. Data load
        x = np.load('x_(7727,10,4068).npy')
        y = np.load('y_(7727,1).npy')
        
        # 2. LSTM model
        def lstm_model():

            seed_num = 42 # model_seed
            tf.random.set_seed(seed_num)

            lstm = Sequential()
            lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
            lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
            lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
            lstm.add(Dropout(0.2))
            lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
            lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
            lstm.add(Dropout(0.2))
            lstm.add(Dense(units=1, activation='sigmoid'))

            return lstm
        
        # 3-1. Best model saving
        from tensorflow.keras.callbacks import ModelCheckpoint
        import os

        MODEL_SAVE_FOLDER_PATH = f'./model2/{name}'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)
                
        data={}
        sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)
        
        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]
            
            # 3-2. Best model saving
            model_path = MODEL_SAVE_FOLDER_PATH + f'/{name}_seed={seed}_'+'{epoch:02d}'+'_val_acc:{val_acc:.4f}.hdf5'

            cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                            verbose=1, save_best_only=True)
            early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=False)

            
            model = lstm_model()
            model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
            model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep,
                      callbacks=[early_stop, cb_checkpoint], shuffle=False)

            # 5. accuracy checking 
            from pathlib import Path
            paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
            best_model_path = str(paths)
            print(best_model_path)

            from keras.models import load_model
            best_model = load_model(best_model_path) 
            y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
            y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
            
            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0
            
            acc = accuracy_score(y_test, y_pred_test)
            data[seed]=acc
            print(f'정확도 : {acc}, seed_num = {seed}')

        df = pd.DataFrame.from_dict(data, orient='index')
        print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    

SyntaxError: invalid syntax (<ipython-input-21-fb41568d47eb>, line 63)

## val_acc / LSTM_notallfit_noearly(name, ep)

In [18]:
## 수정 

from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

import random
import sklearn
from tqdm.notebook import tqdm

def LSTM_notallfit_noearly(name,ep):
    with tf.device('/device:GPU:0'):
        
        # 1. Data load
        x = np.load('x_(7727,10,4068).npy')
        y = np.load('y_(7727,1).npy')
        
        # 2. LSTM model
        def lstm_model():

            seed_num = 42 # model_seed
            tf.random.set_seed(seed_num)

            lstm = Sequential()
            lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
            lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
            lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
            lstm.add(Dropout(0.2))
            lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
            lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
            lstm.add(Dropout(0.2))
            lstm.add(Dense(units=1, activation='sigmoid'))

            return lstm
        
        # 3-1. Best model saving
        from tensorflow.keras.callbacks import ModelCheckpoint
        import os

        MODEL_SAVE_FOLDER_PATH = f'./model2/{name}'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)
                
        data={}
        sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)
        
        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]
            
            # 3-2. Best model saving
            model_path = MODEL_SAVE_FOLDER_PATH + f'/{name}_seed={seed}_'+'{epoch:02d}'+'_val_acc:{val_acc:.4f}.hdf5'

            cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                            verbose=1, save_best_only=True)
            model = lstm_model()
            model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
            model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep,  callbacks=[cb_checkpoint], shuffle=False)

            # 5. accuracy checking 
            from pathlib import Path
            paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
            best_model_path = str(paths)
            print(best_model_path)

            from keras.models import load_model
            best_model = load_model(best_model_path) 
            y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
            y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
            
            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0
            
            acc = accuracy_score(y_test, y_pred_test)
            data[seed]=acc
            print(f'정확도 : {acc}, seed_num = {seed}')

        df = pd.DataFrame.from_dict(data, orient='index')
        print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    

## val_loss / LSTM_notallfit_early_loss(name,ep)

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

import random
import sklearn
from tqdm.notebook import tqdm

def LSTM_notallfit_early_loss(name,ep):
    with tf.device('/device:GPU:0'):
        
        # 1. Data load
        x = np.load('x_(7727,10,4068).npy')
        y = np.load('y_(7727,1).npy')
        
        # 2. LSTM model
        def lstm_model():

            seed_num = 42 # model_seed
            tf.random.set_seed(seed_num)

            lstm = Sequential()
            lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
            lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
            lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
            lstm.add(Dropout(0.2))
            lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
            lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
            lstm.add(Dropout(0.2))
            lstm.add(Dense(units=1, activation='sigmoid'))

            return lstm
        
        # 3-1. Best model saving
        from tensorflow.keras.callbacks import ModelCheckpoint
        import os

        MODEL_SAVE_FOLDER_PATH = f'./model2/{name}'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)
                
        data={}
        sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)
        
        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]
            
            # 3-2. Best model saving
            model_path = MODEL_SAVE_FOLDER_PATH + f'/{name}_seed={seed}_'+'{epoch:02d}'+'_val_loss:{val_loss:.4f}.hdf5'

            cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                            verbose=1, save_best_only=True)
            early_stop = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=False)

            
            model = lstm_model()
            model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
            model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep,
                      callbacks=[early_stop, cb_checkpoint], shuffle=False)

            # 5. accuracy checking 
            from pathlib import Path
            paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
            best_model_path = str(paths)
            print(best_model_path)

            from keras.models import load_model
            best_model = load_model(best_model_path) 
            y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
            y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
            
            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0
            
            acc = accuracy_score(y_test, y_pred_test)
            data[seed]=acc
            print(f'정확도 : {acc}, seed_num = {seed}')

        df = pd.DataFrame.from_dict(data, orient='index')
        print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    

# LSTM_new

## not all fit

### (76) earlystop / val_acc

In [22]:
LSTM_notallfit_early('notallfit_early',500)

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 6s 81ms/step - loss: 0.6753 - acc: 0.6004 - val_loss: 0.6704 - val_acc: 0.6054

Epoch 00001: val_acc improved from -inf to 0.60543, saving model to ./model2/notallfit_early/notallfit_early_seed=0_01_val_acc:0.6054.hdf5
Epoch 2/500
37/37 [==============================] - 3s 71ms/step - loss: 0.6696 - acc: 0.6097 - val_loss: 0.6541 - val_acc: 0.6054

Epoch 00002: val_acc did not improve from 0.60543
Epoch 3/500
37/37 [==============================] - 3s 70ms/step - loss: 0.5885 - acc: 0.6783 - val_loss: 0.5440 - val_acc: 0.7296

Epoch 00003: val_acc improved from 0.60543 to 0.72962, saving model to ./model2/notallfit_early/notallfit_early_seed=0_03_val_acc:0.7296.hdf5
Epoch 4/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4601 - acc: 0.7968 - val_loss: 0.5492 - val_acc: 0.7393

Epoch 00004: val_acc improved from 0.72962 to 0.73933, saving model to ./model2/notallfit_early/notallfit_early_seed=0_04_val_acc:0.7393.

37/37 [==============================] - 3s 69ms/step - loss: 0.0659 - acc: 0.9851 - val_loss: 1.1405 - val_acc: 0.7206

Epoch 00040: val_acc did not improve from 0.73933
Epoch 41/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0644 - acc: 0.9860 - val_loss: 1.1760 - val_acc: 0.7180

Epoch 00041: val_acc did not improve from 0.73933
Epoch 42/500
37/37 [==============================] - 2s 67ms/step - loss: 0.0612 - acc: 0.9845 - val_loss: 1.2037 - val_acc: 0.7160

Epoch 00042: val_acc did not improve from 0.73933
Epoch 43/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0640 - acc: 0.9858 - val_loss: 1.2810 - val_acc: 0.7057

Epoch 00043: val_acc did not improve from 0.73933
Epoch 44/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0678 - acc: 0.9832 - val_loss: 1.2573 - val_acc: 0.7076

Epoch 00044: val_acc did not improve from 0.73933
Epoch 45/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0650 - acc: 0.98

37/37 [==============================] - 3s 70ms/step - loss: 0.1270 - acc: 0.9648 - val_loss: 0.8472 - val_acc: 0.7400

Epoch 00021: val_acc did not improve from 0.77684
Epoch 22/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1177 - acc: 0.9676 - val_loss: 0.8841 - val_acc: 0.7432

Epoch 00022: val_acc did not improve from 0.77684
Epoch 23/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1110 - acc: 0.9726 - val_loss: 0.9200 - val_acc: 0.7406

Epoch 00023: val_acc did not improve from 0.77684
Epoch 24/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1048 - acc: 0.9741 - val_loss: 0.9557 - val_acc: 0.7400

Epoch 00024: val_acc did not improve from 0.77684
Epoch 25/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0996 - acc: 0.9750 - val_loss: 0.9730 - val_acc: 0.7374

Epoch 00025: val_acc did not improve from 0.77684
Epoch 26/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1049 - acc: 0.97

37/37 [==============================] - 3s 68ms/step - loss: 0.6718 - acc: 0.6011 - val_loss: 0.6474 - val_acc: 0.6171

Epoch 00002: val_acc did not improve from 0.61708
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.5979 - acc: 0.6783 - val_loss: 0.5097 - val_acc: 0.7691

Epoch 00003: val_acc improved from 0.61708 to 0.76908, saving model to ./model2/notallfit_early/notallfit_early_seed=2_03_val_acc:0.7691.hdf5
Epoch 4/500
37/37 [==============================] - 2s 68ms/step - loss: 0.4724 - acc: 0.7879 - val_loss: 0.4721 - val_acc: 0.7814

Epoch 00004: val_acc improved from 0.76908 to 0.78137, saving model to ./model2/notallfit_early/notallfit_early_seed=2_04_val_acc:0.7814.hdf5
Epoch 5/500
37/37 [==============================] - 3s 69ms/step - loss: 0.3942 - acc: 0.8373 - val_loss: 0.4870 - val_acc: 0.7865

Epoch 00005: val_acc improved from 0.78137 to 0.78655, saving model to ./model2/notallfit_early/notallfit_early_seed=2_05_val_acc:0.7865.hdf5
Epoc

...

37/37 [==============================] - 3s 68ms/step - loss: 0.0617 - acc: 0.9860 - val_loss: 1.1388 - val_acc: 0.7154

Epoch 00050: val_acc did not improve from 0.76585
Epoch 51/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0595 - acc: 0.9860 - val_loss: 1.1522 - val_acc: 0.7206

Epoch 00051: val_acc did not improve from 0.76585
Epoch 52/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0569 - acc: 0.9875 - val_loss: 1.1760 - val_acc: 0.7128

Epoch 00052: val_acc did not improve from 0.76585
Epoch 53/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0585 - acc: 0.9868 - val_loss: 1.1756 - val_acc: 0.7135

Epoch 00053: val_acc did not improve from 0.76585
Epoch 54/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0569 - acc: 0.9877 - val_loss: 1.1894 - val_acc: 0.7135

Epoch 00054: val_acc did not improve from 0.76585
Epoch 55/500
37/37 [==============================] - 2s 68ms/step - loss: 0.0551 - acc: 0.98

37/37 [==============================] - 2s 67ms/step - loss: 0.1300 - acc: 0.9629 - val_loss: 0.7559 - val_acc: 0.7633

Epoch 00028: val_acc did not improve from 0.76908
Epoch 29/500
37/37 [==============================] - 3s 68ms/step - loss: 0.1154 - acc: 0.9681 - val_loss: 0.7967 - val_acc: 0.7633

Epoch 00029: val_acc did not improve from 0.76908
Epoch 30/500
37/37 [==============================] - 3s 68ms/step - loss: 0.1097 - acc: 0.9715 - val_loss: 0.8238 - val_acc: 0.7620

Epoch 00030: val_acc did not improve from 0.76908
Epoch 31/500
37/37 [==============================] - 2s 68ms/step - loss: 0.1020 - acc: 0.9743 - val_loss: 0.8571 - val_acc: 0.7600

Epoch 00031: val_acc did not improve from 0.76908
Epoch 32/500
37/37 [==============================] - 2s 67ms/step - loss: 0.0947 - acc: 0.9763 - val_loss: 0.8700 - val_acc: 0.7581

Epoch 00032: val_acc did not improve from 0.76908
Epoch 33/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0910 - acc: 0.97

37/37 [==============================] - 3s 70ms/step - loss: 0.6703 - acc: 0.6026 - val_loss: 0.6374 - val_acc: 0.6229

Epoch 00002: val_acc did not improve from 0.62290
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.5878 - acc: 0.6850 - val_loss: 0.5077 - val_acc: 0.7671

Epoch 00003: val_acc improved from 0.62290 to 0.76714, saving model to ./model2/notallfit_early/notallfit_early_seed=8_03_val_acc:0.7671.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4665 - acc: 0.7927 - val_loss: 0.4919 - val_acc: 0.7633

Epoch 00004: val_acc did not improve from 0.76714
Epoch 5/500
37/37 [==============================] - 3s 68ms/step - loss: 0.4138 - acc: 0.8231 - val_loss: 0.5509 - val_acc: 0.7225

Epoch 00005: val_acc did not improve from 0.76714
Epoch 6/500
37/37 [==============================] - 3s 68ms/step - loss: 0.3739 - acc: 0.8455 - val_loss: 0.5220 - val_acc: 0.7542

Epoch 00006: val_acc did not improve from 0.76714
Epoch

37/37 [==============================] - 3s 70ms/step - loss: 0.0511 - acc: 0.9877 - val_loss: 1.0160 - val_acc: 0.7329

Epoch 00046: val_acc did not improve from 0.76714
Epoch 47/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0511 - acc: 0.9879 - val_loss: 1.0447 - val_acc: 0.7329

Epoch 00047: val_acc did not improve from 0.76714
Epoch 48/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0462 - acc: 0.9892 - val_loss: 1.0713 - val_acc: 0.7290

Epoch 00048: val_acc did not improve from 0.76714
Epoch 49/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0432 - acc: 0.9905 - val_loss: 1.1003 - val_acc: 0.7303

Epoch 00049: val_acc did not improve from 0.76714
Epoch 50/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0437 - acc: 0.9901 - val_loss: 1.1202 - val_acc: 0.7309

Epoch 00050: val_acc did not improve from 0.76714
Epoch 51/500
37/37 [==============================] - 2s 67ms/step - loss: 0.0409 - acc: 0.99

37/37 [==============================] - 3s 70ms/step - loss: 0.0918 - acc: 0.9769 - val_loss: 0.9613 - val_acc: 0.7296

Epoch 00028: val_acc did not improve from 0.76391
Epoch 29/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0909 - acc: 0.9776 - val_loss: 0.9701 - val_acc: 0.7277

Epoch 00029: val_acc did not improve from 0.76391
Epoch 30/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0855 - acc: 0.9789 - val_loss: 0.9993 - val_acc: 0.7303

Epoch 00030: val_acc did not improve from 0.76391
Epoch 31/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0831 - acc: 0.9797 - val_loss: 1.0139 - val_acc: 0.7309

Epoch 00031: val_acc did not improve from 0.76391
Epoch 32/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0779 - acc: 0.9827 - val_loss: 1.0388 - val_acc: 0.7329

Epoch 00032: val_acc did not improve from 0.76391
Epoch 33/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0754 - acc: 0.98

### (76.2) earlystop / val_loss

In [27]:
LSTM_notallfit_early_loss('notallfit_early_loss',500)

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 6s 79ms/step - loss: 0.6753 - acc: 0.6004 - val_loss: 0.6704 - val_acc: 0.6054

Epoch 00001: val_loss improved from inf to 0.67041, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=0_01_val_loss:0.6704.hdf5
Epoch 2/500
37/37 [==============================] - 2s 67ms/step - loss: 0.6696 - acc: 0.6097 - val_loss: 0.6541 - val_acc: 0.6054

Epoch 00002: val_loss improved from 0.67041 to 0.65412, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=0_02_val_loss:0.6541.hdf5
Epoch 3/500
37/37 [==============================] - 2s 67ms/step - loss: 0.5884 - acc: 0.6785 - val_loss: 0.5440 - val_acc: 0.7296

Epoch 00003: val_loss improved from 0.65412 to 0.54401, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=0_03_val_loss:0.5440.hdf5
Epoch 4/500
37/37 [==============================] - 2s 67ms/step - loss: 0.4600 - acc: 0.7970 - val_loss: 0.5487 - val_acc: 0.7387

Epoch 0000

정확도 : 0.7613195342820182, seed_num = 0
Epoch 1/500
37/37 [==============================] - 6s 80ms/step - loss: 0.6777 - acc: 0.5929 - val_loss: 0.6674 - val_acc: 0.6132

Epoch 00001: val_loss improved from inf to 0.66743, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=1_01_val_loss:0.6674.hdf5
Epoch 2/500
37/37 [==============================] - 3s 69ms/step - loss: 0.6690 - acc: 0.6080 - val_loss: 0.6467 - val_acc: 0.6132

Epoch 00002: val_loss improved from 0.66743 to 0.64668, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=1_02_val_loss:0.6467.hdf5
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.6049 - acc: 0.6574 - val_loss: 0.5301 - val_acc: 0.7652

Epoch 00003: val_loss improved from 0.64668 to 0.53007, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=1_03_val_loss:0.5301.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5009 - acc: 0.7657 - val_los

정확도 : 0.7723156532988357, seed_num = 1
Epoch 1/500
37/37 [==============================] - 6s 79ms/step - loss: 0.6766 - acc: 0.6011 - val_loss: 0.6684 - val_acc: 0.6171

Epoch 00001: val_loss improved from inf to 0.66844, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=2_01_val_loss:0.6684.hdf5
Epoch 2/500
37/37 [==============================] - 3s 68ms/step - loss: 0.6718 - acc: 0.6011 - val_loss: 0.6474 - val_acc: 0.6171

Epoch 00002: val_loss improved from 0.66844 to 0.64736, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=2_02_val_loss:0.6474.hdf5
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5979 - acc: 0.6781 - val_loss: 0.5098 - val_acc: 0.7665

Epoch 00003: val_loss improved from 0.64736 to 0.50978, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=2_03_val_loss:0.5098.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4724 - acc: 0.7879 - val_los

정확도 : 0.7580853816300129, seed_num = 2
Epoch 1/500
37/37 [==============================] - 6s 80ms/step - loss: 0.6799 - acc: 0.5970 - val_loss: 0.6633 - val_acc: 0.6274

Epoch 00001: val_loss improved from inf to 0.66328, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=3_01_val_loss:0.6633.hdf5
Epoch 2/500
37/37 [==============================] - 3s 69ms/step - loss: 0.6695 - acc: 0.6006 - val_loss: 0.6342 - val_acc: 0.62740s - loss: 0.6724 

Epoch 00002: val_loss improved from 0.66328 to 0.63424, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=3_02_val_loss:0.6342.hdf5
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.5818 - acc: 0.7038 - val_loss: 0.5395 - val_acc: 0.7309

Epoch 00003: val_loss improved from 0.63424 to 0.53946, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=3_03_val_loss:0.5395.hdf5
Epoch 4/500
37/37 [==============================] - 3s 68ms/step - loss: 0.4778 - acc

...

Epoch 1/500
37/37 [==============================] - 6s 81ms/step - loss: 0.6751 - acc: 0.6045 - val_loss: 0.6792 - val_acc: 0.5957

Epoch 00001: val_loss improved from inf to 0.67919, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=5_01_val_loss:0.6792.hdf5
Epoch 2/500
37/37 [==============================] - 3s 69ms/step - loss: 0.6666 - acc: 0.6121 - val_loss: 0.6625 - val_acc: 0.5957

Epoch 00002: val_loss improved from 0.67919 to 0.66254, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=5_02_val_loss:0.6625.hdf5
Epoch 3/500
37/37 [==============================] - 2s 68ms/step - loss: 0.5978 - acc: 0.6762 - val_loss: 0.5389 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.66254 to 0.53888, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=5_03_val_loss:0.5389.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4739 - acc: 0.7905 - val_loss: 0.5111 - val_acc: 0.7613

Epoch 0000

Epoch 1/500
37/37 [==============================] - 6s 82ms/step - loss: 0.6730 - acc: 0.6080 - val_loss: 0.6758 - val_acc: 0.5964

Epoch 00001: val_loss improved from inf to 0.67581, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=6_01_val_loss:0.6758.hdf5
Epoch 2/500
37/37 [==============================] - 3s 68ms/step - loss: 0.6629 - acc: 0.6149 - val_loss: 0.6409 - val_acc: 0.5964

Epoch 00002: val_loss improved from 0.67581 to 0.64090, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=6_02_val_loss:0.6409.hdf5
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5602 - acc: 0.7148 - val_loss: 0.5344 - val_acc: 0.7257

Epoch 00003: val_loss improved from 0.64090 to 0.53441, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=6_03_val_loss:0.5344.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4680 - acc: 0.7901 - val_loss: 0.5145 - val_acc: 0.7613

Epoch 0000

Epoch 1/500
37/37 [==============================] - 6s 78ms/step - loss: 0.6775 - acc: 0.5933 - val_loss: 0.6675 - val_acc: 0.6138

Epoch 00001: val_loss improved from inf to 0.66747, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=7_01_val_loss:0.6675.hdf5
Epoch 2/500
37/37 [==============================] - 3s 68ms/step - loss: 0.6803 - acc: 0.5976 - val_loss: 0.6660 - val_acc: 0.6138

Epoch 00002: val_loss improved from 0.66747 to 0.66600, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=7_02_val_loss:0.6660.hdf5
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.6677 - acc: 0.6123 - val_loss: 0.6363 - val_acc: 0.6138

Epoch 00003: val_loss improved from 0.66600 to 0.63634, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=7_03_val_loss:0.6363.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5826 - acc: 0.6787 - val_loss: 0.5192 - val_acc: 0.7439

Epoch 0000

Epoch 1/500
37/37 [==============================] - 6s 80ms/step - loss: 0.6783 - acc: 0.5927 - val_loss: 0.6622 - val_acc: 0.6229

Epoch 00001: val_loss improved from inf to 0.66219, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=8_01_val_loss:0.6622.hdf5
Epoch 2/500
37/37 [==============================] - 3s 69ms/step - loss: 0.6703 - acc: 0.6026 - val_loss: 0.6374 - val_acc: 0.6229

Epoch 00002: val_loss improved from 0.66219 to 0.63737, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=8_02_val_loss:0.6374.hdf5
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.5878 - acc: 0.6852 - val_loss: 0.5076 - val_acc: 0.7658

Epoch 00003: val_loss improved from 0.63737 to 0.50764, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=8_03_val_loss:0.5076.hdf5
Epoch 4/500
37/37 [==============================] - 3s 68ms/step - loss: 0.4665 - acc: 0.7929 - val_loss: 0.4920 - val_acc: 0.7639

Epoch 0000

Epoch 1/500
37/37 [==============================] - 6s 80ms/step - loss: 0.6713 - acc: 0.6153 - val_loss: 0.6805 - val_acc: 0.5899

Epoch 00001: val_loss improved from inf to 0.68054, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=9_01_val_loss:0.6805.hdf5
Epoch 2/500
37/37 [==============================] - 2s 68ms/step - loss: 0.6612 - acc: 0.6142 - val_loss: 0.6390 - val_acc: 0.5899

Epoch 00002: val_loss improved from 0.68054 to 0.63901, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=9_02_val_loss:0.6390.hdf5
Epoch 3/500
37/37 [==============================] - 2s 67ms/step - loss: 0.5531 - acc: 0.7161 - val_loss: 0.5428 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.63901 to 0.54275, saving model to ./model2/notallfit_early_loss/notallfit_early_loss_seed=9_03_val_loss:0.5428.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4425 - acc: 0.8082 - val_loss: 0.5425 - val_acc: 0.7490

Epoch 0000

### () epoch = 500

In [ ]:
LSTM_notallfit_noearly('notallfit_epoch=500',500)

# LSTM

## all fit

### (89) earlystop / val_acc
- `LSH/model/allfit3_seed42-18-0.7588.hdf5`

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

import random

with tf.device('/device:GPU:0'):
    # ----------------------
    seed_num = 42
    # ----------------------
    random.seed(seed_num)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    # ---------------------
    seed_num = 42 
    # ---------------------
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + 'allfit3_seed42-{epoch:02d}-{val_acc:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                    verbose=1, save_best_only=True)

    early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
    lstm.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 7s 79ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865

Epoch 00001: val_acc improved from -inf to 0.48654, saving model to ./model/allfit3_seed42-01-0.4865.hdf5
Epoch 2/500
46/46 [==============================] - 3s 67ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865

Epoch 00002: val_acc did not improve from 0.48654
Epoch 3/500
46/46 [==============================] - 3s 67ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865

Epoch 00003: val_acc did not improve from 0.48654
Epoch 4/500
46/46 [==============================] - 3s 69ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865

Epoch 00004: val_acc did not improve from 0.48654
Epoch 5/500
46/46 [==============================] - 3s 68ms/step - loss: 0.5325 - acc: 0.7370 - val_loss: 0.5728 - val_acc: 0.7091

Epoch 00005: val_acc improved from 0.48654 to 0.70911, saving model to ./model/

46/46 [==============================] - 3s 69ms/step - loss: 0.0837 - acc: 0.9810 - val_loss: 1.0598 - val_acc: 0.7220

Epoch 00040: val_acc did not improve from 0.75880
Epoch 41/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0816 - acc: 0.9812 - val_loss: 1.0806 - val_acc: 0.7200

Epoch 00041: val_acc did not improve from 0.75880
Epoch 42/500
46/46 [==============================] - 3s 69ms/step - loss: 0.0805 - acc: 0.9807 - val_loss: 1.0899 - val_acc: 0.7226

Epoch 00042: val_acc did not improve from 0.75880
Epoch 43/500
46/46 [==============================] - 3s 68ms/step - loss: 0.0786 - acc: 0.9833 - val_loss: 1.1090 - val_acc: 0.7205

Epoch 00043: val_acc did not improve from 0.75880
Epoch 44/500
46/46 [==============================] - 3s 68ms/step - loss: 0.0775 - acc: 0.9829 - val_loss: 1.1242 - val_acc: 0.7231

Epoch 00044: val_acc did not improve from 0.75880
Epoch 45/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0749 - acc: 0.98

In [3]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/allfit3_seed42-18-0.7588.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

Using TensorFlow backend.


정확도 :0.8783171521035599, seed_num = 0
정확도 :0.8867313915857605, seed_num = 1
정확도 :0.8783171521035599, seed_num = 2
정확도 :0.8957928802588997, seed_num = 3
정확도 :0.888673139158576, seed_num = 4
정확도 :0.8906148867313916, seed_num = 5
정확도 :0.8912621359223301, seed_num = 6
정확도 :0.8951456310679612, seed_num = 7
정확도 :0.9048543689320389, seed_num = 8
정확도 :0.8815533980582524, seed_num = 9
정확도 :0.8957928802588997, seed_num = 10
정확도 :0.8970873786407767, seed_num = 11
정확도 :0.9016181229773462, seed_num = 12
정확도 :0.884789644012945, seed_num = 13
정확도 :0.887378640776699, seed_num = 14
정확도 :0.8964401294498382, seed_num = 15
정확도 :0.9074433656957929, seed_num = 16
정확도 :0.8822006472491909, seed_num = 17
정확도 :0.8828478964401294, seed_num = 18
정확도 :0.8919093851132686, seed_num = 19
정확도 :0.8880258899676375, seed_num = 20
정확도 :0.8776699029126214, seed_num = 21
정확도 :0.8951456310679612, seed_num = 22
정확도 :0.889967637540453, seed_num = 23
정확도 :0.8906148867313916, seed_num = 24
정확도 :0.8996763754045307, seed_num = 25


### (89.7) epoch = 500 / val_acc
- `LSH/model/allfit_ep500/allfit_ep500_seed42-17-0.7619.hdf5`

In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random

with tf.device('/device:GPU:0'):
    # ----------------------
    seed_num = 42
    # ----------------------
    random.seed(seed_num)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    # ---------------------
    seed_num = 42 
    # ---------------------
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    # ---------------------
    name = 'allfit_ep500'
    # ---------------------

    MODEL_SAVE_FOLDER_PATH = f'./model/{name}'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + f'/{name}_seed42-'+'{epoch:02d}'+'-{val_acc:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                    verbose=1, save_best_only=True)

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
    lstm.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 7s 83ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865

Epoch 00001: val_acc improved from -inf to 0.48654, saving model to ./model/allfit_ep500/allfit_ep500_seed42-01-0.4865.hdf5
Epoch 2/500
46/46 [==============================] - 3s 75ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865

Epoch 00002: val_acc did not improve from 0.48654
Epoch 3/500
46/46 [==============================] - 3s 71ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865

Epoch 00003: val_acc did not improve from 0.48654
Epoch 4/500
46/46 [==============================] - 3s 72ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865

Epoch 00004: val_acc did not improve from 0.48654
Epoch 5/500
46/46 [==============================] - 3s 74ms/step - loss: 0.5325 - acc: 0.7374 - val_loss: 0.5728 - val_acc: 0.7091

Epoch 00005: val_acc improved from 0.48654 to 0.70911, saving


Epoch 00083: val_acc did not improve from 0.76190
Epoch 84/500
46/46 [==============================] - 3s 75ms/step - loss: 0.0472 - acc: 0.9919 - val_loss: 1.2751 - val_acc: 0.7200

Epoch 00084: val_acc did not improve from 0.76190
Epoch 85/500
46/46 [==============================] - 3s 74ms/step - loss: 0.0490 - acc: 0.9903 - val_loss: 1.2808 - val_acc: 0.7195

Epoch 00085: val_acc did not improve from 0.76190
Epoch 86/500
46/46 [==============================] - 3s 74ms/step - loss: 0.0502 - acc: 0.9898 - val_loss: 1.2502 - val_acc: 0.7226

Epoch 00086: val_acc did not improve from 0.76190
Epoch 87/500
46/46 [==============================] - 3s 75ms/step - loss: 0.0501 - acc: 0.9902 - val_loss: 1.2585 - val_acc: 0.7184

Epoch 00087: val_acc did not improve from 0.76190
Epoch 88/500
46/46 [==============================] - 3s 72ms/step - loss: 0.0484 - acc: 0.9915 - val_loss: 1.2695 - val_acc: 0.7174

Epoch 00088: val_acc did not improve from 0.76190
Epoch 89/500
46/46 [=========

46/46 [==============================] - 3s 73ms/step - loss: 0.0254 - acc: 0.9955 - val_loss: 1.5062 - val_acc: 0.7277

Epoch 00172: val_acc did not improve from 0.76190
Epoch 173/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0226 - acc: 0.9967 - val_loss: 1.5446 - val_acc: 0.7262

Epoch 00173: val_acc did not improve from 0.76190
Epoch 174/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0223 - acc: 0.9967 - val_loss: 1.5694 - val_acc: 0.7231

Epoch 00174: val_acc did not improve from 0.76190
Epoch 175/500
46/46 [==============================] - 3s 72ms/step - loss: 0.0225 - acc: 0.9965 - val_loss: 1.5872 - val_acc: 0.7231

Epoch 00175: val_acc did not improve from 0.76190
Epoch 176/500
46/46 [==============================] - 3s 75ms/step - loss: 0.0227 - acc: 0.9967 - val_loss: 1.6085 - val_acc: 0.7241

Epoch 00176: val_acc did not improve from 0.76190
Epoch 177/500
46/46 [==============================] - 3s 74ms/step - loss: 0.0218 - acc:

46/46 [==============================] - 3s 74ms/step - loss: 0.0203 - acc: 0.9967 - val_loss: 2.1592 - val_acc: 0.7101

Epoch 00260: val_acc did not improve from 0.76190
Epoch 261/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0205 - acc: 0.9967 - val_loss: 2.1359 - val_acc: 0.7101

Epoch 00261: val_acc did not improve from 0.76190
Epoch 262/500
46/46 [==============================] - 3s 74ms/step - loss: 0.0202 - acc: 0.9967 - val_loss: 2.1562 - val_acc: 0.7076

Epoch 00262: val_acc did not improve from 0.76190
Epoch 263/500
46/46 [==============================] - 3s 72ms/step - loss: 0.0207 - acc: 0.9967 - val_loss: 2.1966 - val_acc: 0.7081

Epoch 00263: val_acc did not improve from 0.76190
Epoch 264/500
46/46 [==============================] - 3s 74ms/step - loss: 0.0281 - acc: 0.9943 - val_loss: 1.8851 - val_acc: 0.7138

Epoch 00264: val_acc did not improve from 0.76190
Epoch 265/500
46/46 [==============================] - 3s 72ms/step - loss: 0.0559 - acc:

46/46 [==============================] - 3s 73ms/step - loss: 0.0163 - acc: 0.9967 - val_loss: 2.0961 - val_acc: 0.7112

Epoch 00348: val_acc did not improve from 0.76190
Epoch 349/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0162 - acc: 0.9967 - val_loss: 2.1006 - val_acc: 0.7122

Epoch 00349: val_acc did not improve from 0.76190
Epoch 350/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0152 - acc: 0.9967 - val_loss: 2.1073 - val_acc: 0.7133

Epoch 00350: val_acc did not improve from 0.76190
Epoch 351/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0147 - acc: 0.9967 - val_loss: 2.1220 - val_acc: 0.7122

Epoch 00351: val_acc did not improve from 0.76190
Epoch 352/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0151 - acc: 0.9967 - val_loss: 2.1330 - val_acc: 0.7117

Epoch 00352: val_acc did not improve from 0.76190
Epoch 353/500
46/46 [==============================] - 3s 75ms/step - loss: 0.0153 - acc:

46/46 [==============================] - 3s 72ms/step - loss: 0.0098 - acc: 0.9967 - val_loss: 2.3518 - val_acc: 0.7169

Epoch 00436: val_acc did not improve from 0.76190
Epoch 437/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0095 - acc: 0.9969 - val_loss: 2.3557 - val_acc: 0.7148

Epoch 00437: val_acc did not improve from 0.76190
Epoch 438/500
46/46 [==============================] - 3s 72ms/step - loss: 0.0099 - acc: 0.9967 - val_loss: 2.3598 - val_acc: 0.7164

Epoch 00438: val_acc did not improve from 0.76190
Epoch 439/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0095 - acc: 0.9967 - val_loss: 2.3649 - val_acc: 0.7153

Epoch 00439: val_acc did not improve from 0.76190
Epoch 440/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0091 - acc: 0.9967 - val_loss: 2.3658 - val_acc: 0.7153

Epoch 00440: val_acc did not improve from 0.76190
Epoch 441/500
46/46 [==============================] - 3s 73ms/step - loss: 0.0095 - acc:

In [10]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):
    
    # ---------------------
    name = 'allfit_ep500'
    # ---------------------

    MODEL_SAVE_FOLDER_PATH = f'./model/{name}'

    from pathlib import Path
    paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
    best_model_path = str(paths)

    from keras.models import load_model
    best_model = load_model(best_model_path) 


    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.8938511326860842, seed_num = 0
정확도 :0.8977346278317152, seed_num = 1
정확도 :0.8919093851132686, seed_num = 2
정확도 :0.8951456310679612, seed_num = 3
정확도 :0.8932038834951457, seed_num = 4
정확도 :0.8938511326860842, seed_num = 5
정확도 :0.8983818770226537, seed_num = 6
정확도 :0.9016181229773462, seed_num = 7
정확도 :0.9048543689320389, seed_num = 8
정확도 :0.8932038834951457, seed_num = 9
정확도 :0.9009708737864077, seed_num = 10
정확도 :0.9029126213592233, seed_num = 11
정확도 :0.8970873786407767, seed_num = 12
정확도 :0.8951456310679612, seed_num = 13
정확도 :0.8932038834951457, seed_num = 14
정확도 :0.8951456310679612, seed_num = 15
정확도 :0.9100323624595469, seed_num = 16
정확도 :0.8854368932038835, seed_num = 17
정확도 :0.8919093851132686, seed_num = 18
정확도 :0.8977346278317152, seed_num = 19
정확도 :0.9003236245954692, seed_num = 20
정확도 :0.889967637540453, seed_num = 21
정확도 :0.8951456310679612, seed_num = 22
정확도 :0.9042071197411004, seed_num = 23
정확도 :0.8925566343042072, seed_num = 24
정확도 :0.9080906148867314, seed_num = 

## not all fit

### (82.7) earlystop / val_loss

In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '4068_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 6s 83ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66548, saving model to ./model/4068_seed42-01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 3s 72ms/step - loss: 0.6691 - acc: 0.6089 - val_loss: 0.6488 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66548 to 0.64879, saving model to ./model/4068_seed42-02-0.6488.hdf5
Epoch 3/500
37/37 [==============================] - 3s 72ms/step - loss: 0.6013 - acc: 0.6708 - val_loss: 0.5525 - val_acc: 0.7342

Epoch 00003: val_loss improved from 0.64879 to 0.55246, saving model to ./model/4068_seed42-03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 2s 67ms/step - loss: 0.4873 - acc: 0.7778 - val_loss: 0.5079 - val_acc: 0.7600

Epoch 00004: val_loss improved from 0.55246 to 0.50789, saving model to ./model/4068_seed42-04-0.5079.hdf5
Epoch 5/500
37/37 [=========================

In [8]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/4068_seed42-05-0.5024.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.8213592233009709, seed_num = 0
정확도 :0.8368932038834952, seed_num = 1
정확도 :0.8245954692556634, seed_num = 2
정확도 :0.8375404530744337, seed_num = 3
정확도 :0.8284789644012945, seed_num = 4
정확도 :0.8407766990291262, seed_num = 5
정확도 :0.8116504854368932, seed_num = 6
정확도 :0.8181229773462784, seed_num = 7
정확도 :0.827831715210356, seed_num = 8
정확도 :0.8310679611650486, seed_num = 9
정확도 :0.8271844660194175, seed_num = 10
정확도 :0.8330097087378641, seed_num = 11
정확도 :0.8349514563106796, seed_num = 12
정확도 :0.8323624595469256, seed_num = 13
정확도 :0.8394822006472492, seed_num = 14
정확도 :0.8330097087378641, seed_num = 15
정확도 :0.8368932038834952, seed_num = 16
정확도 :0.8187702265372169, seed_num = 17
정확도 :0.8233009708737864, seed_num = 18
정확도 :0.8362459546925566, seed_num = 19
정확도 :0.8330097087378641, seed_num = 20
정확도 :0.8129449838187702, seed_num = 21
정확도 :0.8252427184466019, seed_num = 22
정확도 :0.8071197411003236, seed_num = 23
정확도 :0.8349514563106796, seed_num = 24
정확도 :0.8446601941747572, seed_num = 

### (82.6) epoch = 500 / val_loss

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'LSTM_4068_noearly_{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 6s 83ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66548, saving model to ./model/LSTM_4068_noearly_01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6691 - acc: 0.6089 - val_loss: 0.6488 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66548 to 0.64878, saving model to ./model/LSTM_4068_noearly_02-0.6488.hdf5
Epoch 3/500
37/37 [==============================] - 3s 71ms/step - loss: 0.6012 - acc: 0.6704 - val_loss: 0.5525 - val_acc: 0.7342

Epoch 00003: val_loss improved from 0.64878 to 0.55252, saving model to ./model/LSTM_4068_noearly_03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4872 - acc: 0.7780 - val_loss: 0.5080 - val_acc: 0.7607

Epoch 00004: val_loss improved from 0.55252 to 0.50803, saving model to ./model/LSTM_4068_noearly_04-0.5080.hdf5
Epoch 5/500
37/37 [=

37/37 [==============================] - 3s 71ms/step - loss: 0.0588 - acc: 0.9862 - val_loss: 1.2148 - val_acc: 0.7212

Epoch 00084: val_loss did not improve from 0.50262
Epoch 85/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0497 - acc: 0.9899 - val_loss: 1.1666 - val_acc: 0.7322

Epoch 00085: val_loss did not improve from 0.50262
Epoch 86/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0486 - acc: 0.9905 - val_loss: 1.1760 - val_acc: 0.7296

Epoch 00086: val_loss did not improve from 0.50262
Epoch 87/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0467 - acc: 0.9918 - val_loss: 1.1816 - val_acc: 0.7348

Epoch 00087: val_loss did not improve from 0.50262
Epoch 88/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0473 - acc: 0.9918 - val_loss: 1.2217 - val_acc: 0.7335

Epoch 00088: val_loss did not improve from 0.50262
Epoch 89/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0489 - acc:

37/37 [==============================] - 3s 71ms/step - loss: 0.0339 - acc: 0.9948 - val_loss: 1.3699 - val_acc: 0.7335

Epoch 00172: val_loss did not improve from 0.50262
Epoch 173/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0321 - acc: 0.9948 - val_loss: 1.3764 - val_acc: 0.7303

Epoch 00173: val_loss did not improve from 0.50262
Epoch 174/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0335 - acc: 0.9948 - val_loss: 1.3772 - val_acc: 0.7303

Epoch 00174: val_loss did not improve from 0.50262
Epoch 175/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0332 - acc: 0.9946 - val_loss: 1.3973 - val_acc: 0.7264

Epoch 00175: val_loss did not improve from 0.50262
Epoch 176/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0337 - acc: 0.9948 - val_loss: 1.3832 - val_acc: 0.7296

Epoch 00176: val_loss did not improve from 0.50262
Epoch 177/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0334 -

37/37 [==============================] - 3s 72ms/step - loss: 0.0312 - acc: 0.9948 - val_loss: 1.4894 - val_acc: 0.7128

Epoch 00260: val_loss did not improve from 0.50262
Epoch 261/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0328 - acc: 0.9948 - val_loss: 1.4886 - val_acc: 0.7135

Epoch 00261: val_loss did not improve from 0.50262
Epoch 262/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0313 - acc: 0.9948 - val_loss: 1.4926 - val_acc: 0.7115

Epoch 00262: val_loss did not improve from 0.50262
Epoch 263/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0319 - acc: 0.9948 - val_loss: 1.4987 - val_acc: 0.7115

Epoch 00263: val_loss did not improve from 0.50262
Epoch 264/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0318 - acc: 0.9948 - val_loss: 1.4951 - val_acc: 0.7109

Epoch 00264: val_loss did not improve from 0.50262
Epoch 265/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0328 -

37/37 [==============================] - 3s 71ms/step - loss: 0.0288 - acc: 0.9950 - val_loss: 1.5097 - val_acc: 0.7141

Epoch 00348: val_loss did not improve from 0.50262
Epoch 349/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0299 - acc: 0.9950 - val_loss: 1.5164 - val_acc: 0.7147

Epoch 00349: val_loss did not improve from 0.50262
Epoch 350/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0292 - acc: 0.9950 - val_loss: 1.5096 - val_acc: 0.7135

Epoch 00350: val_loss did not improve from 0.50262
Epoch 351/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0288 - acc: 0.9950 - val_loss: 1.5114 - val_acc: 0.7128

Epoch 00351: val_loss did not improve from 0.50262
Epoch 352/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0299 - acc: 0.9950 - val_loss: 1.5151 - val_acc: 0.7154

Epoch 00352: val_loss did not improve from 0.50262
Epoch 353/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0303 -

37/37 [==============================] - 3s 71ms/step - loss: 0.0251 - acc: 0.9950 - val_loss: 1.6525 - val_acc: 0.7063

Epoch 00436: val_loss did not improve from 0.50262
Epoch 437/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0251 - acc: 0.9950 - val_loss: 1.6500 - val_acc: 0.7083

Epoch 00437: val_loss did not improve from 0.50262
Epoch 438/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0260 - acc: 0.9950 - val_loss: 1.6523 - val_acc: 0.7063

Epoch 00438: val_loss did not improve from 0.50262
Epoch 439/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0248 - acc: 0.9950 - val_loss: 1.6560 - val_acc: 0.7076

Epoch 00439: val_loss did not improve from 0.50262
Epoch 440/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0268 - acc: 0.9950 - val_loss: 1.6514 - val_acc: 0.7076

Epoch 00440: val_loss did not improve from 0.50262
Epoch 441/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0257 -